## Imagen을 통한 이미지 생성 및 처리

Feedback : shins777@gmail.com. 

이 Colab은 구글의 Image 생성, 수정등을 위한 솔루션인 Imagen에 대한 예제입니다.  
* Image generation
* Image Editing
* Image Captioning
* Visual QnA

현재(2024년 3월 8일기준) Imagen 2.0 버전은 특정 고객만 사용할 수 있는 Private preview 상태입니다.  참고하세요. 만일 해당 기능을 사용하시기 원하시면 구글에 문의 해주세요.  
구글은 텍스트와 마찬가지로 이미지 생성에서 있어서 "모델에 사용된 training 데이터에 대해 고객이 책임지지 않음" 과 "해당 모델에 의해서 생성된 컨텐트에 대해서 고객이 책임지지 않는다는" 두가지의 고객의 면책을 보장합니다.  
자세한 내용은 아래 블로그 참고하세요.
* https://cloud.google.com/blog/products/ai-machine-learning/protecting-customers-with-generative-ai-indemnification

In [ ]:
%pip install --upgrade --quiet matplotlib

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
# from google.colab import auth
# auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [ ]:
project="ai-hangsik"
location="asia-northeast3"
imagen_model_name="imagegeneration@005"

### Vertex AI 초기화

In [ ]:
import vertexai
from vertexai.preview.vision_models import ImageGenerationModel

vertexai.init(project=project, location=location)
generation_model = ImageGenerationModel.from_pretrained(imagen_model_name)


#### 생성된 이미지 배치를 위한 함수.

In [ ]:
import math
import matplotlib.pyplot as plt

def display_images_in_grid(images):
    nrows = math.ceil(len(images) / 4)
    ncols = min(len(images) + 1, 4)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 6))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i]._pil_image)
            ax.set_aspect("equal")
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.axis("off")
    plt.tight_layout()
    plt.show()

### 이미지 생성

이미지를 생성하기 위해서는 마치 사진을 찍는것처럼 Prompting을 해주시는것이 좋습니다.   
* 아래는 Image genearation 에서 좀더 효율적으로 Prompting 하는 방법을 나타내주는 메뉴얼 입니다.  
    * https://cloud.google.com/vertex-ai/docs/generative-ai/image/img-gen-prompt-guide?hl=ko

In [ ]:
prompt = "A beautiful ice prince with silver hair and It has 'Star' written on his forehead, fantasy concept art style"

response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42,
)

display_images_in_grid(response.images)


### Image captioning
* Image captioning 기능은 이미지 내의 정보를 생성해주는 기능입니다.  
* https://cloud.google.com/vertex-ai/generative-ai/docs/image/image-captioning?hl=ko
* https://cloud.google.com/vertex-ai/generative-ai/docs/image/quickstart-vcap-vqa-console?hl=ko

In [ ]:
from vertexai.preview.vision_models import ImageCaptioningModel

image_captioning_model = ImageCaptioningModel.from_pretrained("imagetext@001")

In [ ]:
! gsutil cp "gs://github-repo/img/vision/google-cloud-next.jpeg" .

In [ ]:
from vertexai.preview.vision_models import Image

# Load the image file as Image object
cloud_next_image = Image.load_from_file("google-cloud-next.jpeg")
cloud_next_image.show()

In [ ]:
# Get a caption from the image
image_captioning_model.get_captions(
    image=cloud_next_image,
)

In [ ]:
# Get 3 captions from the image
image_captioning_model.get_captions(
    image=cloud_next_image,
    number_of_results=3,
    language="en",

)

### Imange Question and Answer
시각적인 정보를 기반으로 QnA를 할 수도 있습니다.
* https://cloud.google.com/vertex-ai/generative-ai/docs/image/visual-question-answering?hl=ko



In [ ]:
from vertexai.preview.vision_models import ImageQnAModel

image_qna_model = ImageQnAModel.from_pretrained("imagetext@001")

In [ ]:
# Download an image from Google Cloud Storage

! gsutil cp "gs://github-repo/img/vision/google-cloud-next.jpeg" .

In [ ]:
from vertexai.preview.vision_models import Image

# Load the image file as Image object
cloud_next_image = Image.load_from_file("google-cloud-next.jpeg")
cloud_next_image.show()

In [ ]:
# Ask a question about the image
image_qna_model.ask_question(
    image=cloud_next_image, question="What is happening in this image?"
)

In [ ]:
# Get 3 answers from the image
image_qna_model.ask_question(
    image=cloud_next_image,
    question="What are the people in the image doing?",
    number_of_results=3,
)